# Installations

In [ ]:
!pip install AudioConverter

In [ ]:
!pip install deepspeed==0.8.3

# Imports

In [1]:
import os

In [8]:
import tarfile

In [27]:
from IPython.display import Audio

# Clone the repo and install dependencies

In [ ]:
!git clone --recurse-submodules https://github.com/enhuiz/vall-e.git

In [ ]:
%cd vall-e

In [ ]:
!pip install -e .

In [ ]:
%cd ..

# Download and process the librilight data

In [ ]:
!wget https://dl.fbaipublicfiles.com/librilight/data/librispeech_finetuning.tgz

In [9]:
 # open file
file = tarfile.open('librispeech_finetuning.tgz')

# extracting file
file.extractall('./')

file.close()

In [16]:
!mkdir vall-e/data/libri

In [ ]:
for k in os.listdir('librispeech_finetuning/9h/clean'):
    for j in os.listdir('librispeech_finetuning/9h/'+'clean'):
        for z in os.listdir('librispeech_finetuning/9h/clean/'+j):
            for i in os.listdir('librispeech_finetuning/9h/clean/'+j+'/'+z):
                os.rename('librispeech_finetuning/9h/clean/'+j+'/'+z+'/'+i, f'vall-e/data/libri/{i}')
        
    break
lst = []
for i in os.listdir('vall-e/data/libri'):
    try:
        if 'trans' in i:
            with open(f'vall-e/data/libri/{i}') as text_file:
                for row in text_file:
                    z = row.split('-')
                    name = z[0]+'-'+z[1]+ '-' + z[2].split(' ')[0]
                    text = " ".join(z[2].split(' ')[1:])
                    # print(name, text)
                    lst.append([name, text])

                        
    except:
        None      

for i in lst:
    try:
        with open('vall-e/data/libri/'+i[0]+'.txt', 'x') as file:
            # print(i[1])
            file.write(i[1])
    except:
        with open('vall-e/data/libri/'+i[0]+'.txt', 'w+') as file:
            # print(i[1])
            file.write(i[1])

for i in sorted(os.listdir('vall-e/data/libri')):
    if i.split('.')[1] == 'txt':
        # print(i)
        print('.'.join([i.split('.')[0],'normalized',i.split('.')[1]]))
        os.rename('vall-e/data/libri/'+i, 'vall-e/data/libri/'+'.'.join([i.split('.')[0],'normalized',i.split('.')[1]]))

In [19]:
!audioconvert convert vall-e/data/libri/ vall-e/data/libri --output-format .wav

[ INFO    ] Starting conversion of vall-e/data/libri/.
[ SUCCESS ] See vall-e/data/libri for converted audio.


# Quantize the recordings and generate phonemes

In [21]:
%cd vall-e

/content/vall-e


In [22]:
!python -m vall_e.emb.qnt data/libri

  0% 0/1222 [00:00<?, ?it/s]Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th

  0% 0.00/88.9M [00:00<?, ?B/s]
  9% 7.92M/88.9M [00:00<00:01, 83.1MB/s]
 28% 25.1M/88.9M [00:00<00:00, 140MB/s] 
 47% 41.8M/88.9M [00:00<00:00, 156MB/s]
 66% 58.7M/88.9M [00:00<00:00, 164MB/s]
100% 88.9M/88.9M [00:00<00:00, 158MB/s]
100% 1222/1222 [01:37<00:00, 12.56it/s]


In [23]:
!python -m vall_e.emb.g2p data/libri

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
100% 1222/1222 [00:10<00:00, 119.63it/s]


# Train the AR model

- Modify the config files as needed

In [44]:
!mkdir -p zoo

In [ ]:
!python -m vall_e.train yaml=config/test/ar.yml

In [47]:
!python -m vall_e.export zoo/ar.pt yaml=config/test/ar.yml

[2023-05-22 04:07:56,372] [INFO] [comm.py:652:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2023-05-22 04:07:58,258] [INFO] [logging.py:93:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu118/fused_adam/build.ninja...
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module fused_adam...
Time to load fused_adam op: 0.10143852233886719 seconds
[2023-05-22 04:07:59,218] [INFO] [logging.py:93:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adam as basic optimizer
[2023-05-22 04:07:59,221] [INFO] [logging.py:93:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam
[2023-05-22 04:07:59,221] [INFO] [logging.py:93:log_dist] 

# Train the NAR model

In [ ]:
!python -m vall_e.train yaml=config/test/nar.yml

In [45]:
!python -m vall_e.export zoo/nar.pt yaml=config/test/nar.yml

[2023-05-22 03:38:50,504] [INFO] [comm.py:652:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2023-05-22 03:38:52,308] [INFO] [logging.py:93:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu118/fused_adam/build.ninja...
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module fused_adam...
Time to load fused_adam op: 0.09243226051330566 seconds
[2023-05-22 03:38:53,237] [INFO] [logging.py:93:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adam as basic optimizer
[2023-05-22 03:38:53,239] [INFO] [logging.py:93:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam
[2023-05-22 03:38:53,239] [INFO] [logging.py:93:log_dist] 

# Generate a new sample

In [61]:
!python -m vall_e 'hello world' data/test/test.wav toy.wav

 20% 199/1000 [00:03<00:14, 54.74it/s]
toy.wav saved.


In [62]:
Audio('toy.wav')

# Utility

In [25]:
!ls

'/content/vall-e'

In [ ]:
!zip -r data/libri.zip data/libri/

In [ ]:
for f in os.listdir('data/libri'):
  print('size: '+ str(os.path.getsize('data/libri/'+f)))